In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
try:
    import open3d as o3d
except ImportError as e:
    install = input(f'{e}, Do you want to install it? [Y/n]')
    if install == 'Y':
        import sys
        !{sys.executable} -m pip install open3d
        import open3d as o3d
        print(o3d.__version__)

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_dblquad, elementwise_circquad
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              colormap_from_array, save_fig, fig_config)
from utils import *

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# Ear model

## Dataset

In [ ]:
# full dataset

df = load_ear_data('tm', 60)
df = clean_df(df)
df

In [ ]:
# conversion to numpys

xyz = export_pcd(df)
E, H = export_fields(df)

In [ ]:
# visualize ear model in matplotlib

view_config = {
    'zoom': 0.69999999999999996,
    'front': [0.92231085610160646, 0.17546582462905541, 0.34431733779228646],
    'lookat': [71.236805645281521, 22.531933429935712, -8.12589641784127],
    'up': [-0.16595758534247468, 0.98447554162242001, -0.057148821637356101],
}
cframe, pcd = visualize_ear(xyz, show=False, **view_config)
color = get_imcolors([cframe, pcd], view_config)

fig_config()
fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(color, origin='upper');

# fname = os.path.join('figures', 'ear_model')
# save_fig(fig, fname=fname)

## EM dosimetry

In [ ]:
# extracting components of the power density vector field

Sx, Sy, Sz = poynting_vector(E, H)

In [ ]:
# visualization of the absolute value of the power density

fig_config(scaler=2)
S = abs(np.sqrt(Sx ** 2 + Sy ** 2 + Sz ** 2))
S_label = r'$1/2$ $|\vec{E}\times\vec{H}^{*}|$ [W/m2]'
fig, ax = scatter_3d({'z [mm]': xyz[:, 2],
                      'x [mm]': xyz[:, 0],
                      'y [mm]': xyz[:, 1],
                       S_label: S},
                     elev=[10], azim=[120])

In [ ]:
# visualization of the real part of the power density normal to the surface

fig_config(scaler=2)
n = estimate_normals(xyz, knn=30, fast=True)
Sr = abs(Sx.real * n[:, 0] + Sy.real * n[:, 1] + Sz.real * n[:, 2])
Sr_label = r'$1/2$ $\Re{[\vec{E}\times\vec{H}^{*}]} \cdot \vec{n}$ [W/m2]'
fig, ax = scatter_3d({'z [mm]': xyz[:, 2],
                      'x [mm]': xyz[:, 0],
                      'y [mm]': xyz[:, 1],
                      Sr_label: Sr},
                      elev=[10], azim=[180])

In [ ]:
# removing the sharp-edged region of the model and
# visualization of the real part of the power density normal to the surface

fig_config(scaler=2)
crop_idxs = np.where(xyz[:, 0] > 67)[0]
xyz_crop = xyz[crop_idxs]
Sx_crop, Sy_crop, Sz_crop = Sx[crop_idxs], Sy[crop_idxs], Sz[crop_idxs]
n_crop = estimate_normals(xyz_crop, knn=30, fast=True)
Sr_crop = abs(Sx_crop.real * n_crop[:, 0]
              + Sy_crop.real * n_crop[:, 1]
              + Sz_crop.real * n_crop[:, 2])
fig, ax = scatter_3d({'z [mm]': xyz_crop[:, 2],
                      'x [mm]': xyz_crop[:, 0],
                      'y [mm]': xyz_crop[:, 1],
                      Sr_label: Sr_crop},
                     elev=[10], azim=[180], figsize=(10, 10))

In [ ]:
# define coordinate frame in open3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
pcd.paint_uniform_color([0.5, 0.5, 0.5])
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10,
                                                           origin=center)
# o3d.visualization.draw_geometries([cframe, pcd])

In [ ]:
# translate the coordinates of the model to have the center at (0, 0, 0)

xyz_t = np.c_[xyz[:, 0] - center[0],
              xyz[:, 1] - center[1],
              xyz[:, 2] - center[2]]
pcd_t = o3d.geometry.PointCloud()
pcd_t.points = o3d.utility.Vector3dVector(xyz_t)
center_t = pcd_t.get_center()
pcd_t.paint_uniform_color([0.5, 0.5, 0.5])
cframe_t = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10,
                                                             origin=center_t)
# o3d.visualization.draw_geometries([cframe_t, pcd_t])

## zy-plane

In [ ]:
# extract the points visible from the yz-plane point of view

diameter = np.linalg.norm(pcd_t.get_max_bound() - pcd_t.get_min_bound())
radius = 10 ** 5.5
camera = [diameter, 0, 0]

_, pt_map = pcd_t.hidden_point_removal(camera, radius)

xyz_t_zy = xyz_t[pt_map]
Sr_t_zy = Sr[pt_map]
pcd_t_zy = o3d.geometry.PointCloud()
pcd_t_zy.points = o3d.utility.Vector3dVector(xyz_t_zy)
pcd_t_zy.colors = o3d.utility.Vector3dVector(colormap_from_array(Sr_t_zy))
# o3d.visualization.draw_geometries([cframe_t, pcd_t_zy])

In [ ]:
# visualization of the real part of the power density normal to the zy-surface

fig_config(scaler=1.5)
fig, ax = scatter_2d({'z [mm]': xyz_t_zy[:, 2],
                      'y [mm]': xyz_t_zy[:, 1],
                      Sr_label: Sr_t_zy},
                     s=1);

In [ ]:
# extract rectangular averaging suface area

center = [-12.5, 12.5]
edge_length = 10
origin, idx_rect = export_rect_idx(xyz=xyz_t_zy,
                                   center=center,
                                   edge_length=edge_length,
                                   view='zy')
xyz_rect = xyz_t_zy[idx_rect]
Sr_rect = Sr_t_zy[idx_rect]

In [ ]:
# visualization of the real part of the power density normal to the extracted
# rectangular averaging surface area of 1x1 cm2

fig_config(scaler=1.5)
fig, ax = scatter_2d({'z [mm]': xyz_t_zy[:, 2],
                      'y [mm]': xyz_t_zy[:, 1],
                      Sr_label: Sr_t_zy}, s=1)
patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=3)
avg_surf = ax.add_patch(patch_rect);

# zoom in
fig_config(scaler=1.5)
fig, ax = scatter_2d({'z [mm]': xyz_rect[:, 2],
                      'y [mm]': xyz_rect[:, 1],
                      Sr_label: Sr_rect}, s=20);

# computing the average absorbed power density
area = edge_length ** 2
APD_rect = elementwise_dblquad(points=np.c_[xyz_rect[:, 2], xyz_rect[:, 1]],
                               values=Sr_rect,
                               degree=11) / area
print(f'APD = {APD_rect:.7f} W/m2')

In [ ]:
# extract circular averaging suface area

radius = np.sqrt(area / np.pi)
idx_circ = export_circ_idx(xyz=xyz_t_zy,
                           center=center,
                           radius=radius,
                           view='zy')
xyz_circ = xyz_t_zy[idx_circ]
Sr_circ = Sr_t_zy[idx_circ]

In [ ]:
# visualization of the real part of the power density normal to the extracted
# circular averaging surface area of 1 cm2

fig_config(scaler=1.5)
fig, ax = scatter_2d({'z [mm]': xyz_t_zy[:, 2],
                      'y [mm]': xyz_t_zy[:, 1],
                      Sr_label: Sr_t_zy}, s=1)
patch_circ = Circle(center, radius, fc='None', lw=3)
avg_surf = ax.add_patch(patch_circ);

# zoom in
fig_config(scaler=1.5)
fig, ax = scatter_2d({'z [mm]': xyz_circ[:, 2],
                      'y [mm]': xyz_circ[:, 1],
                      Sr_label: Sr_circ}, s=20);

# computing the average abosrbed power density
APD_circ = elementwise_circquad(points=np.c_[xyz_circ[:, 2], xyz_circ[:, 1]],
                                values=Sr_circ,
                                radius=radius,
                                center=center,
                                degree=11) / area
print(f'APD = {APD_circ:.7f} W/m2')

In [ ]:
# difference in dB between absorbed power denesities computed on rectangular-
# and circular-shaped averaging surface

APD_diff = diff_in_dB(APD_rect, APD_circ)
print(f'APD_diff = {APD_diff:.7f} dB')

## xy-plane

In [ ]:
# define coordinate frame in open3d for cropped ear model

pcd_crop = o3d.geometry.PointCloud()
pcd_crop.points = o3d.utility.Vector3dVector(xyz_crop)
center_crop = pcd.get_center()
pcd_crop.paint_uniform_color([0.5, 0.5, 0.5]);
# o3d.visualization.draw_geometries([cframe, pcd_crop])

In [ ]:
# translate the coordinates of the model to have the center at (0, 0, 0)

xyz_crop_t = np.c_[xyz_crop[:, 0] - center_crop[0],
                   xyz_crop[:, 1] - center_crop[1],
                   xyz_crop[:, 2] - center_crop[2]]
pcd_crop_t = o3d.geometry.PointCloud()
pcd_crop_t.points = o3d.utility.Vector3dVector(xyz_crop_t)
center_crop_t = pcd_t.get_center()
pcd_crop_t.paint_uniform_color([0.5, 0.5, 0.5])
cframe_crop_t = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10,
                                                                  origin=center_crop_t)
#o3d.visualization.draw_geometries([cframe_crop_t, pcd_crop_t])

In [ ]:
# extract the points visible from the xy-plane point of view

diameter = np.linalg.norm(pcd_crop_t.get_max_bound() - pcd_crop_t.get_min_bound())
radius = 10 ** 5.5
camera = [0, 0, -diameter]

_, pt_map = pcd_crop_t.hidden_point_removal(camera, radius)

xyz_crop_t_xy = xyz_crop_t[pt_map]
Sr_crop_t_xy = Sr_crop[pt_map]
pcd_crop_t_xy = o3d.geometry.PointCloud()
pcd_crop_t_xy.points = o3d.utility.Vector3dVector(xyz_crop_t_xy)
pcd_crop_t_xy.colors = o3d.utility.Vector3dVector(colormap_from_array(Sr_crop_t_xy))
# o3d.visualization.draw_geometries([cframe_crop_t, pcd_crop_t_xy])

In [ ]:
# visualization of the real part of the power density normal to the zy-surface

fig_config(scaler=1.5)
fig, ax = scatter_2d({'x [mm]': xyz_crop_t_xy[:, 0],
                      'y [mm]': xyz_crop_t_xy[:, 1],
                      Sr_label: Sr_crop_t_xy},
                     s=1)

ax.invert_xaxis()

In [ ]:
# extract rectangular averaging suface area

center = [3.1, 3.25]
edge_length = 10
origin, idx_rect = export_rect_idx(xyz=xyz_crop_t_xy,
                                   center=center,
                                   edge_length=edge_length,
                                   view='xy')
xyz_rect = xyz_crop_t_xy[idx_rect]
Sr_rect = Sr_crop_t_xy[idx_rect]

In [ ]:
# visualization of the real part of the power density normal to the extracted
# rectangular averaging surface area of 1x1 cm2

fig_config(latex=True, scaler=1.5)
fig, ax = scatter_2d({'$x$ [mm]': xyz_crop_t_xy[:, 0],
                      '$y$ [mm]': xyz_crop_t_xy[:, 1],
                      Sr_label: Sr_crop_t_xy}, s=1)
patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=3)
avg_surf = ax.add_patch(patch_rect)
ax.invert_xaxis()

fname = os.path.join('figures', PROJECT_NAME, 'te_60_full_rect')
save_fig(fig, fname=fname)

In [ ]:
# zoom in
fig_config(latex=True, scaler=1.5)
fig, ax = scatter_2d({'x [mm]': xyz_rect[:, 0],
                      'y [mm]': xyz_rect[:, 1],
                      Sr_label: Sr_rect}, s=20)
ax.invert_xaxis()

fname = os.path.join('figures', PROJECT_NAME, 'te_60_int_rect')
save_fig(fig, fname=fname)

In [ ]:
# computing the average absorbed power density
area = edge_length ** 2
APD_rect = elementwise_dblquad(points=np.c_[xyz_rect[:, 0], xyz_rect[:, 1]],
                               values=Sr_rect,
                               degree=11) / area
print(f'APD = {APD_rect:.7f} W/m2')

In [ ]:
# extract circular averaging suface area

radius = np.sqrt(area / np.pi)
idx_circ = export_circ_idx(xyz=xyz_crop_t_xy,
                           center=center,
                           radius=radius,
                           view='xy')
xyz_circ = xyz_crop_t_xy[idx_circ]
Sr_circ = Sr_crop_t_xy[idx_circ]

In [ ]:
# visualization of the real part of the power density normal to the extracted
# circular averaging surface area of 1 cm2

fig_config(latex=True, scaler=1)
fig, ax = scatter_2d({'x [mm]': xyz_crop_t_xy[:, 0],
                      'y [mm]': xyz_crop_t_xy[:, 1],
                      Sr_label: Sr_crop_t_xy}, s=1)
patch_circ = Circle(center, radius, fc='None', lw=3)
avg_surf = ax.add_patch(patch_circ)
ax.invert_xaxis()

fname = os.path.join('figures', PROJECT_NAME, 'te_60_full_circ')
save_fig(fig, fname=fname)

# zoom in
fig_config(latex=True, scaler=1.5)
fig, ax = scatter_2d({'x [mm]': xyz_circ[:, 0],
                      'y [mm]': xyz_circ[:, 1],
                      Sr_label: Sr_circ}, s=20)
ax.invert_xaxis()

fname = os.path.join('figures', PROJECT_NAME, 'te_60_int_circ')
save_fig(fig, fname=fname)

# computing the average abosrbed power density
APD_circ = elementwise_circquad(points=np.c_[xyz_circ[:, 0], xyz_circ[:, 1]],
                                values=Sr_circ,
                                radius=radius,
                                center=center,
                                degree=11) / area
print(f'APD = {APD_circ:.7f} W/m2')

In [ ]:
# difference in dB between absorbed power denesities computed on rectangular-
# and circular-shaped averaging surface

APD_diff = diff_in_dB(APD_rect, APD_circ)
print(f'APD_diff = {APD_diff:.7f} dB')

## Integration domains

In [ ]:
# circular integration domain - set up scheme

import quadpy
scheme = quadpy.s2.get_good_scheme(12)

total_area = np.pi
flt = np.vectorize(float)
pts = flt(scheme.points.T)
weights = flt(scheme.weights.T)
radii = np.sqrt(abs(weights) / np.sum(weights) * total_area / np.pi)
colors = ['gray' if weight >= 0 else 'r' for weight in weights]

In [ ]:
# visualize circular integration domain

fig_config(scaler=1.5)
fig = plt.figure()
ax = fig.add_subplot()
ax.axis("equal")
ax.set_axis_off()
disk = Circle((0, 0), 1, ec='k', fc='None')
ax.add_patch(disk)
for pt, radius, color in zip(pts, radii, colors):
    plt.plot([pt[0]], [pt[1]], linestyle="None", marker=".", color=color)
    circ = Circle((pt[0], pt[1]), radius, color=color, alpha=0.7, linewidth=0)
    ax.add_patch(circ)
    
# fname = os.path.join('figures', PROJECT_NAME, 'circ_int_scheme')
# save_fig(fig, fname=fname)

In [ ]:
# rectangular integration domain - set up scheme

deg = 6
a, b = -0.5, 0.5
x, w = np.polynomial.legendre.leggauss(deg)
x = 0.5 * (x + 1.) * (b - a) + a
w = 0.5 * w * (b - a)
Xx, Xy = np.meshgrid(x, x)
Wx, Wy = np.meshgrid(w, w)
pts = np.c_[Xx.ravel(), Xy.ravel()]
weights = np.c_[Wx.ravel(), Wy.ravel()]
weights_norm = np.linalg.norm(weights, axis=1, ord=sum(abs(x)**2)**(1./2))

total_area = 1.2
radii = np.sqrt(abs(weights_norm) / np.sum(weights_norm) * total_area / np.pi)
colors = ['gray' if weight >= 0 else 'r' for weight in weights_norm]

In [ ]:
# visualize rectangular integration domain

fig_config(scaler=1.5)
fig = plt.figure()
ax = fig.add_subplot()
ax.axis("equal")
ax.set_axis_off()
rect = Rectangle((-0.5, -0.5), 1, 1, ec='k', fc='None')
ax.add_patch(rect)
for pt, radius, color in zip(pts, radii, colors):
    plt.plot([pt[0]], [pt[1]], linestyle="None", marker=".", color=color)
    circ = Circle((pt[0], pt[1]), radius, color=color, alpha=0.7, linewidth=0)
    ax.add_patch(circ)
    
# fname = os.path.join('figures', PROJECT_NAME, 'rect_int_scheme')
# save_fig(fig, fname=fname)